In [2]:
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from icecream import ic

In [3]:
class FabricPhysicalConstants:
    rho= 2300 #kg/m^3 
    cp= 750 # J/(kg-K) 
    k=0.8 # W/(m-K)
    h = 4 # W/(m-K)
    T0= 293.15 # K 

    alpha = k/(cp*rho) # thermal diffusivity

In [35]:
class TransientWallConduction:
    def __init__(self, f:FabricPhysicalConstants):
        self.f = f
        # x vals init in example => th = 0.10 m, dt = 0.010 m 
        thickness = 0.10 
        self.dx = 0.010
        self.x_vals = np.arange(start=0, stop=thickness, step=self.dx)
        self.M = len(self.x_vals)

        # time vals init 
        self.dt = 15 # seconds also from example 
        period = self.dt * 4 * 60/20 # 1 hour/ 
        self.times = np.arange(start=0, stop=period , step=self.dt)
        self.N = len(self.times)
        # TODO resolve difference between this time formulation and that of indoor air, also use time stamps...

        # initialize N*M matrix - time * x nodes 
        self.Ttx = np.zeros((self.N, self.M))
        self.Ttx[0,:] = f.T0

        # time constant
        # # TODO check tau  
        self.tau = self.f.alpha * self.dt / (self.dx**2)
        

    def calc_boundary_nodes(self, i):
        # TODO beta should be different based on side of wall 
        beta = self.f.h * self.dx / self.f.k
        eq = lambda Tself, Tnb: (1 -2*self.tau - 2*self.tau*beta)*Tself + 2*self.tau*Tnb + 2*self.tau*beta
        
        T0 = eq(self.Ttx[i,0], self.Ttx[i, 1] )
        TM = eq(self.Ttx[i,self.M-1], self.Ttx[i, self.M-2] )

        return T0, TM

    def calc_interior_nodes(self, i):
        Tint =  np.zeros(self.M)
        row = self.Ttx[i,:] 
        for m in range(self.M):
            m = m + 1 # avoid first and last nodes 
            if m < len(self.x_vals) - 1:
                Tint[m] = self.tau*(row[m - 1] + row[m + 1]) + (1 - 2*self.tau)*row[m]
        
        assert Tint[0] == 0 and Tint[self.M-1] == 0 

        return Tint 
    
    def calc_Tx_at_t(self, i):
        if i < self.N - 1:
            self.Ttx[i+1, :] = self.calc_interior_nodes(i)
            self.Ttx[i+1, 0], self.Ttx[i+1, self.M-1] = self.calc_boundary_nodes(i)

            return self.Ttx[i+1, :]

    def calc_all(self):
        for i in range(self.N):
            self.calc_Tx_at_t(i)

        return self.Ttx

In [36]:
t = TransientWallConduction(FabricPhysicalConstants())
Txt = t.calc_all()

In [37]:
fig = go.Figure()

for time, x_at_time in enumerate(t.Ttx):
    fig.add_trace(go.Scatter(x=t.x_vals, y=x_at_time, mode='lines+markers', name=time))
fig.update_layout(xaxis_title='Distance Along Wall (X)',
                  yaxis_title='Temperature (ºC)',
                  title='Heat Conduction in Wall')


In [3]:
def calc_boundary(i):
    # TODO beta should be different based on side of wall 
    T0 = (1 -2*tau - 2*tau*beta)*Txt[i,0] + 2*tau*Txt[i, 1] + 2*tau*beta

    TM = (1 -2*tau - 2*tau*beta)*Txt[i,M-1] + 2*tau*Txt[i, M-2] + 2*tau*beta

    return T0, TM

    # TODO make class!


In [4]:
def calc_transient_wall_conduction(f:FabricPhysicalConstants):
    # x vals init 
    
    # in example => th = 0.10 m, dt = 0.010 m 
    thickness = 0.10 
    dx = 0.010
    x_vals = np.arange(start=0, stop=thickness, step=dx)
    M = len(x_vals)

    # time vals init 
    dt = 15 # seconds also from example 
    period = dt * 4 * 60/20 # 1 hour/ 
    times = np.arange(start=0, stop=period , step=dt)
    N = len(times)
    # TODO resolve difference between this time formulation and that of indoor air => ALSO need to do check of tau based on Cengel 

    # initialize big array 
    # # TODO should be called Ttx because t by x 
    Txt = np.zeros((N, M))
    Txt[0,:] = f.T0


    # initialize intermediate arrays for interior node calculation => # TODO put in own function 
    Ta =  np.zeros(M)
    Tb = np.zeros(M)
    ic(len(Ta), len(Tb), Txt.shape)

    # beta => for convection 
    beta = f.h * dx / f.k
    tau = f.alpha * dt / (dx**2)

    for i, _ in enumerate(times):
        row = Txt[i,:] # row at a given timestep
        for m, _ in enumerate(Txt[i,:]): # iterate over the interior nodes
            m = m + 1
            if m < len(x_vals) - 1:
                Ta[m] = row[m - 1] + row[m + 1]
                Tb[m] = row[m]

            if i < len(times) - 1:
                Txt[i+1, :] = Ta*tau + Tb*(1 - 2*tau)
                # update boundary conditions 
                T0, TM = calc_boundary(i)
                Txt[i+1, 0] = T0
                Txt[i+1, M-1] = TM

    return Txt

In [5]:
Txt = calc_transient_wall_conduction(FabricPhysicalConstants())

ic| len(Ta): 10, len(Tb): 10, Txt.shape: (12, 10)


NameError: name 'tau' is not defined

In [ ]:
fig = go.Figure()

for t, x_at_time in enumerate(Txt):
    fig.add_trace(go.Scatter(x=x_vals, y=x_at_time, mode='lines+markers', name=t))
fig.update_layout(xaxis_title='Distance Along Wall (X)',
                  yaxis_title='Temperature (ºC)',
                  title='Heat Conduction in Wall')

# TODO => temp at wall edges should be outdoor and indorr temps respectively... 